In [1]:
%matplotlib inline
# Tests for some CCA stuff on pig data.
import itertools
import numpy as np
import os
import torch
from torch import nn

from dataprocessing import predictive_maintenance_datasets
from models import \
    embeddings, greedy_multi_view_rl, greedy_single_view_rl,\
    naive_multi_view_rl, naive_single_view_rl, ovr_mcca_embeddings,\
    robust_multi_ae, torch_models, ts_fourier_featurization
from synthetic import multimodal_systems as ms
from tests.test_greedy_mvrl import default_GMVRL_config
from tests.test_mv_pig_data import \
    plot_heatmap, default_NGSRL_config, default_RMAE_config,\
    aggregate_multipig_data, rescale, split_data, make_subset_list, error_func,\
    all_subset_accuracy
from utils import torch_utils, utils, time_series_utils

try:
  import matplotlib.pyplot as plt
  MPL_AVAILABLE = True
except ImportError:
  MPL_AVAILABLE = False


import IPython


# Plotting funcs:
def plot_windows(
    tvals, labels, ndisp, title, nwin, wsize, ax=None, shuffle=True):
  plot_ts = []
  for win in tvals:
    plot_ts.append(win.reshape(-1, win.shape[-1]))

  if nwin is not None and nwin > 0:
    ndisp = nwin * wsize

  if shuffle and ndisp > 0:
    IPython.embed()
    shuffle_inds = [np.random.permutation(p.shape[0])[:ndisp] for p in plot_ts]
    plot_ts = [win[inds] for win, inds in zip(plot_ts, shuffle_inds)]

  if ndisp > 0:
    plot_ts = [win[:ndisp] for win in plot_ts]

  ntsteps = plot_ts[0].shape[0]

  ax = plt if ax is None else ax

  for win, lbl in zip(plot_ts, labels):
    ax.plot(win, label=lbl)
  # ax.plot(tv_plot, color='b', label="Ground Truth")
  # ax.plot(op_plot, color='r', label="Predicted")
  if title:
    try:
      ax.title(title, fontsize=30)
      ax.xticks(fontsize=15)
      ax.yticks(fontsize=15)
      ax.legend(fontsize=15)
    except TypeError:
      ax.set_title(title, fontsize=10)
      ax.legend()
      # ax.set_xticks(fontsize=5)
      # ax.set_yticks(fontsize=5)
    # if nwin is not None and nwin > 0:

  win_x = wsize
  while win_x < ntsteps:
    ax.axvline(x = win_x, ls="--")
    win_x += wsize


def split_into_windows(data, window_size, shuffle=True):
  ids, ts, feats, ys = data["ids"], data["ts"], data["features"], data["y"]

  window_data = {key: [] for key in data}
  # Go over the t-series of each unit
  idx = 0
  for u_id, u_ts, u_ft, u_y in zip(ids, ts, feats, ys):
    if u_ft.shape[0] < window_size:
      print("Skipping %s. Not enough data for a window." % (u_id,))
      continue
    u_ts_ft = np.c_[u_ts.reshape(-1, 1), u_ft]
    try:
      w_ts_ft = time_series_utils.split_ts_into_windows(
          u_ts_ft, window_size, ignore_rest=False, shuffle=shuffle)
    except Exception as e:
      IPython.embed()
      raise e
    idx += 1
    w_ts, w_ft = w_ts_ft[:, :, 0], w_ts_ft[:, :, 1:]
    n_win = w_ft.shape[0]
    w_ids = [u_id] * n_win
    w_ys = [u_y] * n_win

    window_data["ids"].extend(w_ids)
    window_data["y"].extend(w_ys)
    window_data["ts"].append(w_ts)
    window_data["features"].append(w_ft)

  for key in ["ts", "features"]:
    window_data[key] = np.concatenate(window_data[key], axis=0)

  return window_data


_FFT_DIM = 30
_FFT_MODEL = None
_CH_MODELS = None
# Featurization depends only on training data. So this should be called first
# on training data.
def fft_featurize_data(window_data):
  global _FFT_MODEL, _CH_MODELS

  ts, feat = window_data["ts"], window_data["features"]
  window_size = feat.shape[1]

  if _FFT_MODEL is None:
    fft_model_file = os.path.join(
        os.getenv("RESEARCH_DIR"), "tests/saved_models",
        "cmas_fft_feats_ws%i.fart" % window_size)
    config = ts_fourier_featurization.FFConfig(
        ndim=_FFT_DIM, use_imag=False, verbose=True)
    _FFT_MODEL = ts_fourier_featurization.TimeSeriesFourierFeaturizer(config)
    if os.path.exists(fft_model_file):
      print("Loading fft model.")
      _FFT_MODEL.load_from_file(fft_model_file)
      _FFT_MODEL.config.ndim = _FFT_DIM
    else:
      print("Training and saving fft model.")
      _FFT_MODEL.fit(feat, ts)
      _FFT_MODEL.save_to_file(fft_model_file)

    _CH_MODELS = _FFT_MODEL.split_channels_into_models()

  mv_data = {}
  for i, ch_model in enumerate(_CH_MODELS):
    ch_windows = feat[:, :, [i]]
    mv_data[i] = np.squeeze(ch_model.encode(ch_windows))

  return mv_data


def load_cmas_data(window_size=20):
  normalize = True
  dset_type = "all"
  data, misc = predictive_maintenance_datasets.load_cmas(dset_type, normalize)

  # Just need to make sure fft feats are done on train data first.
  window_data = {}
  fft_window_data = {}
  for dset_type in ["train", "test"]:
    wdata = split_into_windows(data[dset_type], window_size)
    fft_window_data[dset_type] = fft_featurize_data(wdata)
    window_data[dset_type] = wdata

  return window_data, fft_window_data


def reconstruct_ts(codes, output_len):
  output_ts = {}
  for k, ch_code in codes.items():
    output_ts[k] = _CH_MODELS[k].decode(ch_code, output_len)

  return output_ts

Mutual information import failed.


In [8]:
class args:
  etype = 1
  wsize = 20
  npts = 1000
  njobs = 3
  max_iters = 250
  num_ss_tfm = 1
  num_lin_tfm = 1
  use_leaky_relu = False
  use_reverse = False
  dist_type = "gaussian"
  use_ar = False
  n_components = 5



In [5]:
window_size = args.wsize
npts = args.npts
win_data, cmas_data = load_cmas_data(window_size=window_size)

# Fit model.
tr_w_ffts = cmas_data["train"]
te_w_ffts = cmas_data["test"]
tr_wdata = win_data["train"]["features"]
te_wdata = win_data["test"]["features"]

dsets = {"Train": tr_wdata, "Test": te_wdata}

config = default_NGSRL_config(sv_type="nn")
config.njobs = None if args.njobs == -1 else args.njobs
if npts > 0:
  tr_w_ffts = {vi: d[:npts] for vi, d in tr_w_ffts.items()}

Loading fft model.
Skipping (4, 141). Not enough data for a window.
Skipping (4, 204). Not enough data for a window.


In [9]:
config.single_view_config.lambda_global = 1e-3
config.single_view_config.lambda_group = 0 # 1e-1
config.single_view_config.sp_eps = 5e-5
config.single_view_config.max_iters = args.max_iters

model = naive_multi_view_rl.NaiveBlockSparseMVRL(config)

In [ ]:
model.fit(tr_w_ffts)

Optimizing for group sparse transforms.
  Solving problems in parallel...
Starting training loop.

Iteration 1 out of 250.
Starting training loop.

Iteration 1 out of 250.
Starting training loop.
Starting training loop.

Iteration 1 out of 250.
Iteration 1 out of 250.
Starting training loop.


Iteration 1 out of 250.Starting training loop.


Iteration 1 out of 250.
Starting training loop.

Iteration 1 out of 250.
Starting training loop.

Iteration 1 out of 250.
Starting training loop.

Iteration 1 out of 250.
Starting training loop.

Iteration 1 out of 250.
Starting training loop.

Iteration 1 out of 250.
Starting training loop.

Iteration 1 out of 250.
Starting training loop.

Iteration 1 out of 250.
Starting training loop.

Iteration 1 out of 250.
Starting training loop.

Iteration 1 out of 250.
Starting training loop.

Iteration 1 out of 250.
Starting training loop.

Iteration 1 out of 250.
Starting training loop.

Iteration 1 out of 250.
Starting training loop.

Iteration 1 out of 


Iteration 6 out of 250.
Loss: 31.08772
Iteration 5 took 8.30s.

Iteration 6 out of 250.
Loss: 29.07557
Iteration 5 took 8.37s.

Iteration 6 out of 250.
Loss: 31.30995
Iteration 5 took 8.46s.

Iteration 6 out of 250.
Loss: 36.32806
Iteration 5 took 8.32s.

Iteration 6 out of 250.
Loss: 82.07980
Iteration 5 took 8.81s.

Iteration 6 out of 250.Loss: 36.13579
Iteration 5 took 9.28s.


Iteration 6 out of 250.
Loss: 29.42975
Iteration 5 took 8.24s.
Loss: 29.80224

Iteration 6 out of 250.Iteration 5 took 9.08s.


Iteration 6 out of 250.
Loss: 30.75704
Iteration 5 took 8.67s.

Iteration 6 out of 250.
Loss: 29.14291
Iteration 6 took 7.99s.

Iteration 7 out of 250.
Loss: 31.28350
Iteration 6 took 8.59s.
Loss: 30.83678

Iteration 7 out of 250.Loss: 28.90873

Iteration 5 took 8.50s.
Iteration 6 took 7.42s.

Iteration 6 out of 250.
Iteration 7 out of 250.

Loss: 29.70120
Iteration 5 took 8.92s.

Iteration 6 out of 250.Loss: 28.46459

Iteration 6 took 7.48s.

Iteration 7 out of 250.
Loss: 30.39234


Iteration 12 out of 250.
Loss: 25.77762
Iteration 11 took 7.43s.
Loss: 26.87428

Iteration 12 out of 250.Loss: 32.55165
Loss: 26.37490

Iteration 10 took 8.64s.
Loss: 25.54540
Loss: 25.62021
Iteration 10 took 7.85s.

Iteration 11 out of 250.Iteration 10 took 9.13s.

Iteration 11 out of 250.Iteration 10 took 9.13s.


Iteration 11 out of 250.Iteration 10 took 9.09s.


Iteration 11 out of 250.

Iteration 11 out of 250.
Loss: 26.19830
Iteration 10 took 9.23s.


Iteration 11 out of 250.
Loss: 24.64914
Iteration 11 took 8.08s.

Iteration 12 out of 250.
Loss: 27.40866
Iteration 11 took 9.04s.

Iteration 12 out of 250.
Loss: 24.40545
Loss: 28.88079
Iteration 11 took 8.29s.
Iteration 11 took 8.49s.

Iteration 12 out of 250.
Iteration 12 out of 250.

Loss: 24.52901
Iteration 11 took 8.85s.

Iteration 12 out of 250.
Loss: 25.15530
Loss: 25.77138
Iteration 11 took 8.49s.
Iteration 10 took 8.13s.

Iteration 12 out of 250.
Iteration 11 out of 250.

Loss: 25.74552
Iteration 11 took 8.89s.

Iteration 

Iteration 16 out of 250.
Loss: 22.94968
Iteration 15 took 8.36s.

Iteration 16 out of 250.
Loss: 22.32551
Iteration 15 took 8.89s.

Iteration 16 out of 250.
Loss: 22.39629
Iteration 15 took 7.79s.

Iteration 16 out of 250.
Loss: 25.65798
Loss: 21.94984
Iteration 16 took 8.84s.
Iteration 15 took 8.64s.

Iteration 16 out of 250.
Iteration 17 out of 250.

Loss: 21.37243
Iteration 16 took 8.72s.
Loss: 21.05457

Iteration 17 out of 250.Iteration 16 took 8.60s.


Iteration 17 out of 250.
Loss: 24.27606
Loss: 80.17871
Iteration 16 took 9.20s.
Iteration 16 took 8.22s.

Iteration 17 out of 250.
Iteration 17 out of 250.

Loss: 21.21565
Iteration 16 took 8.84s.
Loss: 21.98518

Iteration 17 out of 250.Iteration 16 took 7.47s.


Iteration 17 out of 250.Loss: 21.98087

Iteration 16 took 9.46s.
Loss: 22.59206

Iteration 17 out of 250.Iteration 16 took 8.42s.
Loss: 22.19951


Iteration 17 out of 250.Iteration 16 took 8.51s.


Iteration 17 out of 250.
Loss: 28.52785
Iteration 16 took 8.04s.
Loss: 23.21


Iteration 21 out of 250.Iteration 20 took 8.75s.


Iteration 21 out of 250.
Loss: 18.11700
Iteration 21 took 9.08s.

Iteration 22 out of 250.
Loss: 18.57545
Iteration 21 took 8.61s.

Iteration 22 out of 250.
Loss: 19.59362
Iteration 21 took 8.96s.

Iteration 22 out of 250.
Loss: 19.22155
Iteration 21 took 8.23s.
Loss: 19.14425

Iteration 22 out of 250.Iteration 21 took 8.28s.


Iteration 22 out of 250.
Loss: 18.95970
Iteration 21 took 8.46s.

Iteration 22 out of 250.
Loss: 18.37426
Loss: 70.93235
Iteration 21 took 8.52s.
Iteration 21 took 8.63s.

Iteration 22 out of 250.
Iteration 22 out of 250.

Loss: 21.46281
Loss: 19.24067
Loss: 20.51509
Iteration 21 took 8.39s.
Iteration 21 took 8.68s.
Iteration 21 took 8.22s.

Iteration 22 out of 250.
Iteration 22 out of 250.Loss: 19.18585


Iteration 22 out of 250.
Iteration 22 took 7.92s.


Iteration 23 out of 250.
Loss: 18.15907
Iteration 21 took 8.43s.

Iteration 22 out of 250.
Loss: 19.45340
Iteration 22 took 8.48s.

Iteration 23 out of 250.

...

Iteration 206 out of 250.

Loss: 1.86073
Loss: 2.44316
Iteration 204 took 8.67s.
Loss: 2.26848

Iteration 205 out of 250.Iteration 205 took 8.77s.


Iteration 206 out of 250.Iteration 205 took 7.99s.
Loss: 2.05713

Iteration 206 out of 250.
Iteration 205 took 7.70s.


Iteration 206 out of 250.
Loss: 1.97423
Loss: 1.82964
Iteration 207 took 8.33s.
Iteration 205 took 8.83s.

Iteration 208 out of 250.
Iteration 206 out of 250.

Loss: 3.32555
Loss: 3.03216
Iteration 205 took 8.33s.
Iteration 207 took 8.73s.

Iteration 206 out of 250.
Iteration 208 out of 250.Loss: 5.82574


Iteration 206 took 8.20s.

Iteration 207 out of 250.
Loss: 20.00506
Iteration 204 took 8.43s.

Iteration 205 out of 250.Loss: 1.42692

Iteration 207 took 8.65s.

Iteration 208 out of 250.
Loss: 3.43667
Loss: 1.50884
Iteration 204 took 7.92s.
Iteration 205 took 8.03s.

Iteration 205 out of 250.
Iteration 206 out of 250.

Loss: 3.82638
Loss: 2.76484
Iteration 207 took 8.77s.
Iteration 206 took 8.79s.

Iteration 208 out of


Iteration 210 took 9.04s.

Iteration 213 out of 250.Loss: 2.13734

Iteration 211 out of 250.

Iteration 210 took 8.73s.

Iteration 211 out of 250.
Loss: 2.27650
Iteration 210 took 9.32s.

Iteration 211 out of 250.
Loss: 2.37675
Loss: 3.13706
Iteration 211 took 8.62s.
Iteration 210 took 8.69s.

Iteration 212 out of 250.
Iteration 211 out of 250.

Loss: 1.75465
Iteration 209 took 9.18s.

Iteration 210 out of 250.
Loss: 5.61223
Iteration 211 took 8.76s.

Iteration 212 out of 250.
Loss: 1.70342
Iteration 210 took 8.21s.

Iteration 211 out of 250.
Loss: 1.37208
Iteration 210 took 8.71s.

Iteration 211 out of 250.
Loss: 4.18931
Loss: 2.56771
Iteration 210 took 7.66s.
Iteration 211 took 8.63s.

Iteration 212 out of 250.
Iteration 211 out of 250.
Loss: 3.23606

Loss: 19.22530
Iteration 209 took 8.03s.
Iteration 209 took 9.32s.
Loss: 7.33978

Iteration 210 out of 250.
Iteration 210 out of 250.
Iteration 212 took 8.56s.


Iteration 213 out of 250.
Loss: 2.59699
Iteration 211 took 8.99s.

Iterat

Loss: 1.65709
Iteration 214 took 8.76s.

Iteration 215 out of 250.
Loss: 1.73654
Iteration 217 took 9.07s.
Loss: 2.02411

Iteration 218 out of 250.Iteration 215 took 8.52s.


Iteration 216 out of 250.
Loss: 2.24934
Iteration 216 took 9.05s.

Iteration 217 out of 250.
Loss: 1.24283
Loss: 5.39568
Iteration 215 took 8.67s.
Iteration 216 took 8.54s.

Iteration 216 out of 250.
Iteration 217 out of 250.

Loss: 2.41760
Iteration 216 took 8.30s.

Iteration 217 out of 250.
Loss: 4.03694
Loss: 3.04673
Iteration 215 took 8.27s.
Iteration 214 took 7.51s.

Iteration 216 out of 250.
Iteration 215 out of 250.

Loss: 7.16054
Iteration 217 took 8.52s.

Iteration 218 out of 250.Loss: 1.58629

Iteration 215 took 8.73s.

Iteration 216 out of 250.
Loss: 3.42015
Iteration 217 took 8.64s.
Loss: 3.29756

Iteration 218 out of 250.Iteration 216 took 7.72s.
Loss: 2.43033

Iteration 217 out of 250.
Iteration 216 took 8.06s.


Iteration 217 out of 250.
Loss: 4.38952
Iteration 216 took 8.29s.

Iteration 217 out of 

Iteration 221 took 8.12s.

Iteration 222 out of 250.
Loss: 3.88761
Loss: 1.62319
Iteration 220 took 7.94s.
Iteration 222 took 9.03s.

Iteration 221 out of 250.
Iteration 223 out of 250.Loss: 5.19506

Loss: 1.12238

Iteration 220 took 7.75s.
Iteration 221 took 8.05s.

Iteration 221 out of 250.Loss: 2.27474

Iteration 222 out of 250.Iteration 221 took 8.38s.



Iteration 222 out of 250.
Loss: 6.96495
Iteration 222 took 7.10s.

Iteration 223 out of 250.
Loss: 2.86662
Iteration 219 took 8.00s.

Iteration 220 out of 250.
Loss: 3.22661
Loss: 1.48033
Iteration 220 took 8.40s.
Iteration 222 took 8.65s.
Loss: 2.74265

Iteration 221 out of 250.
Iteration 223 out of 250.Iteration 221 took 8.82s.
Loss: 2.52193


Iteration 222 out of 250.
Iteration 223 took 8.17s.
Loss: 3.08552

Iteration 224 out of 250.Iteration 221 took 8.23s.



Iteration 222 out of 250.
Loss: 2.86864
Iteration 223 took 8.07s.

Iteration 224 out of 250.Loss: 1.52528
Loss: 4.24962

Loss: 2.27373
Iteration 221 took 8.37s.
Loss: 1.


Iteration 225 out of 250.
Loss: 4.23909
Iteration 225 took 8.84s.

Iteration 226 out of 250.
Loss: 3.75606
Iteration 225 took 9.30s.
Loss: 2.13902

Iteration 226 out of 250.Iteration 226 took 8.48s.


Iteration 227 out of 250.
Loss: 1.01237
Loss: 1.51638
Iteration 225 took 8.55s.
Iteration 227 took 8.67s.

Iteration 226 out of 250.
Iteration 228 out of 250.

Loss: 1.38296
Iteration 226 took 8.53s.

Iteration 227 out of 250.
Loss: 2.37595
Iteration 228 took 8.66s.
Loss: 0.89442
Iteration 228 took 8.22s.

Iteration 229 out of 250.
Iteration 229 out of 250.

Loss: 1.38458
Loss: 2.57293
Iteration 225 took 8.60s.
Loss: 2.70186

Iteration 226 out of 250.Iteration 228 took 8.91s.
Loss: 1.77956
Iteration 226 took 7.88s.


Iteration 229 out of 250.
Iteration 227 out of 250.Iteration 226 took 7.63s.



Iteration 227 out of 250.Loss: 1.46752
Iteration 225 took 7.88s.


Iteration 226 out of 250.Loss: 2.88428

Loss: 1.97691
Iteration 226 took 8.81s.

Iteration 227 out of 250.Iteration 226 took 8.4